#1) Set-Up of necessairy libraries

In [1]:
import graphlab
from graphlab import SFrame


A newer version of GraphLab Create (v1.8.5) is available! Your current version is v1.7.1.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


# 2) Pull in data from the outside

In [4]:
data =  SFrame.read_csv("turnstile_160319.txt",
                        column_type_hints=[str,str,str,str,str,str,str,str,str,int,int])


PROGRESS: Finished parsing file /Users/Zhanna/Documents/Coding_Projects/Metis/My_work/Challenges/01-benson/turnstile_160319.txt
PROGRESS: Parsing completed. Parsed 191890 lines in 1.05033 secs.


#3) See what features/columns our data has

In [5]:
data.show()

Canvas is updated and available in a tab in the default browser.


In [9]:
data.column_names()

['C/A',
 'UNIT',
 'SCP',
 'STATION',
 'LINENAME',
 'DIVISION',
 'DATE',
 'TIME',
 'DESC',
 'ENTRIES',
 'EXITS']

In [13]:
#data['TIME'] = data['TIME'].astype("datetime.datetime")
#data['DATE '] = data['DATE'].astype("datetime.datetime")

dir(data['DATE'])

['__abs__',
 '__add__',
 '__and__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__div__',
 '__doc__',
 '__eq__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__get_content_identifier__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__is_materialized__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__materialize__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__proxy__',
 '__radd__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmod__',
 '__rmul__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '_count_ngrams',
 '_count_words',
 '_escape_space',
 '_getitem_cache',
 '_save_as_text',
 'all',
 'any',
 'append',
 'apply',
 'argmax',
 'argmin',
 'astype',
 'clip',
 'clip_lower',
 'clip_upper',
 'contains',
 'date_range',
 'datetime_t

#4) See the house prices per zipcode to deduce which zip code tends to have the highest prices

In [16]:
# graphlab.canvas.set_target('ipynb')
sales.show(view='BoxWhisker Plot',x='zipcode',y='prices')

##5) Observe 98039 is the most expensive zip code. Select out a portion of houses which are just this zip code.

In [22]:
houses_98039 = sales[sales['zipcode']=='98039']

In [24]:
houses_98039.show()

In [26]:
houses_98039['price']

dtype: int
Rows: 50
[2950000, 1905000, 875000, 940000, 4000000, 5570000, 937500, 1160000, 1880000, 3418800, 1555000, 1398000, 1320000, 6885000, 865000, 1000000, 2100000, 2575000, 1712750, 787500, 1651000, 1800000, 1550000, 1762000, 1225000, 2750000, 1950000, 2200000, 1260000, 2450000, 3640900, 2271150, 1620000, 3710000, 1200000, 2500000, 2187730, 1450000, 1880000, 2458000, 1350000, 2680000, 1410000, 1920000, 3635000, 2720000, 1950000, 2250000, 2900000, 1865000]

#6) ANSWER: Calculate the mean directly:
#### Note: you could also see this above if you look at the price column and its summary statistics.
#### Here, I calculate it using the subset of data and directly from our sales data.

In [28]:
houses_98039['price'].mean()

2160606.5999999996

In [32]:
sales[sales['zipcode'] == '98039']['price'].mean()

2160606.5999999996

# 7) Select the houses that have ‘sqft_living’ higher than 2000 sqft but no larger than 4000 sqft.

In [34]:
houses_98039_medium = houses_98039[(houses_98039["sqft_living"] > 2000) & (houses_98039["sqft_living"] < 4000)]

In [41]:
print houses_98039_medium.num_rows()
print houses_98039.num_rows()

24
50


#8) ANSWER: Tells you what fraction of houses are in this medium ranger.... 24/50 or 48/100 or 0.48

In [56]:
x =  float(houses_98039_medium.num_rows() / float(houses_98039.num_rows()))
y = float(24/50.000)
print x
print y

0.48
0.48


# 9) Adding a larget set of features.

In [95]:
simple_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'zipcode']

In [104]:
advanced_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'zipcode',
'condition', 'grade','waterfront','view', 'sqft_above','sqft_basement','yr_built', 'yr_renovated','lat', 'long', 'sqft_living15','sqft_lot15', ]

#9.5) Break out the houses in the original sample (not just in zip code 98039) into training and split data.

In [113]:
train_data,test_data = sales.random_split(.8,seed=0)

#10) Create a model for a basic set of features

In [114]:
basic_model = graphlab.linear_regression.create(train_data,target = 'price',features = simple_features,validation_set=None)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 17384
PROGRESS: Number of features          : 6
PROGRESS: Number of unpacked features : 6
PROGRESS: Number of coefficients    : 115
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 0.044390     | 3763208.270523     | 181908.848367 |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: SUCCESS: Optimal solution found.
PROGRESS:


#11) Create a model for an advanced set of features

In [115]:
advanced_model = graphlab.linear_regression.create(train_data,target = 'price',features = advanced_features,validation_set=None)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 17384
PROGRESS: Number of features          : 18
PROGRESS: Number of unpacked features : 18
PROGRESS: Number of coefficients    : 127
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 0.064132     | 3469012.450686     | 154580.940736 |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: SUCCESS: Optimal solution found.
PROGRESS:


#12) Compute the RMSE for each model

In [116]:
basic_model.evaluate(test_data)
basic_rsme = basic_model.evaluate(test_data)['rmse']
print basic_rsme

179542.433313


In [117]:
advanced_model.evaluate(test_data)
advanced_rsme = advanced_model.evaluate(test_data)['rmse']
print advanced_rsme

156831.116802


In [120]:
print basic_model.evaluate(test_data)
print advanced_model.evaluate(test_data)

{'max_error': 3486584.509381705, 'rmse': 179542.4333126903}
{'max_error': 3556849.413858208, 'rmse': 156831.1168021901}


#13) Calculate the difference between the two RMSE

In [121]:
basic_rsme - advanced_rsme

22711.316510500183

#14) We observe that the advanced has a much greater RMSE!